<a href="https://colab.research.google.com/github/deepakri201/SR_for_ProstateX/blob/main/SR_for_ProstateX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creation of DICOM for ProstateX

In this notebook, we create all of the DICOM SRs for ProstateX. The csvs used for the annotations are obtained from TCIA: https://www.cancerimagingarchive.net/collection/prostatex/

The data from the 1st challenge do not include the Gleason Grade Group and only include if each lesion is clinically significant or not: located in google drive for now, later upload as github attachment to download quickly.

The data from the 2nd challenge include the Gleason Grade Group: the csvs for the training lesion info are [here](https://www.cancerimagingarchive.net/wp-content/uploads/ProstateX2-DataInfo-Train.zip), and the testing lesion info is [here](https://www.cancerimagingarchive.net/wp-content/uploads/ProstateX2-DataInfo-Test.zip)

However, the other prostate information remains the same.


Deepa Krishnaswamy

Brigham and Women's Hospital

May 2025

In [1]:
location = 'us-central1'
bucket_sr_directory = "gs://sr_prostatex"

# Parameterization

In [2]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# Environment setup

In [3]:
# Authorize
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 17.5 MB/s eta 0:00:00


In [6]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.2 MB/s eta 0:00:00


In [7]:
!pip install highdicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 32.6 MB/s eta 0:00:00


In [8]:
!pip install idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


In [9]:
import os
import sys
import time
import re

from google.cloud import bigquery
from google.cloud import storage

import nibabel as nib
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt

import json

import pydicom

In [10]:
from pathlib import Path

import pydicom
from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

import highdicom
from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

from highdicom.sr import ImageLibrary
from highdicom.sr import ImageLibraryEntryDescriptors
from highdicom.sr.value_types import Code
from highdicom.sr.value_types import ContentItem
# from highdicom.sr import ContentItem, ValueTypeValues, RelationshipTypeValues
from highdicom.sr.coding import CodedConcept

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [11]:
from idc_index import IDCClient
client = IDCClient.client()

In [12]:
pydicom.__version__

'3.0.1'

In [13]:
highdicom.__version__

'0.27.0'

# Get the metadata tables from IDC

In [14]:
client_bq = bigquery.Client(project=project_name)

In [15]:
query1 = f"""
    SELECT
      findings.dicom_patient_id AS PatientID,
      findings.fid,
      findings.pos,
      findings.zone,
      findings.clinsig,
      images.dcmserdescr AS SeriesDescription,
      images.dcmsernum AS SeriesNumber
    FROM
      `bigquery-public-data.idc_current_clinical.prostatex_findings` as findings
    JOIN
      `bigquery-public-data.idc_current_clinical.prostatex_images` as images
    ON
      findings.dicom_patient_id = images.dicom_patient_id AND
      findings.fid = images.fid AND
      findings.pos = images.pos
    WHERE
      images.dcmserdescr LIKE "%t2_tse_tra%"
      """

query2 = f"""
  SELECT
    findings.dicom_patient_id AS PatientID,
    findings.fid,
    findings.pos,
    findings.zone,
    findings.ggg,
    images.dcmserdescr AS SeriesDescription,
    images.dcmsernum AS SeriesNumber,
    images.dcmseruid AS SeriesInstanceUID
  FROM
    `bigquery-public-data.idc_current_clinical.prostatex_findings2` as findings
  JOIN
    `bigquery-public-data.idc_current_clinical.prostatex_images2` as images
  ON
    findings.dicom_patient_id = images.dicom_patient_id AND
    findings.fid = images.fid AND
    findings.pos = images.pos
  WHERE
    images.dcmserdescr LIKE "%t2_tse_tra%"
  """

job_config = bigquery.QueryJobConfig()
df1 = client_bq.query(query1, job_config=job_config).to_dataframe()
df2 = client_bq.query(query2, job_config=job_config).to_dataframe()

# Then merge to create single df
df = pd.merge(left=df1, right=df2, how="outer", on=['PatientID', 'fid', 'pos', 'zone', 'SeriesDescription', 'SeriesNumber'])

def merge_rows(group):
  # Start with the first row as the base
  row = group.iloc[0].copy()

  # Prefer non-null ggg from any row
  non_null_ggg = group['ggg'].dropna()
  if not non_null_ggg.empty:
    row['ggg'] = non_null_ggg.iloc[0]

  # Prefer non-null dcmseruid, and match its corresponding dcmsernum
  non_null_uid_row = group.dropna(subset=['SeriesInstanceUID'])
  if not non_null_uid_row.empty:
    row['SeriesInstanceUID'] = non_null_uid_row.iloc[0]['SeriesInstanceUID']
    row['SeriesNumber'] = non_null_uid_row.iloc[0]['SeriesNumber']
    row['SeriesDescription'] = non_null_uid_row.iloc[0]['SeriesDescription']

  return row

# Now we need to merge the rows, so that the ggg and clinsig are appropriately combined per pos
# Apply per (dicom_patient_id, pos)
df = df.groupby(['PatientID', 'pos'], as_index=False).apply(merge_rows).reset_index(drop=True)


/tmp/ipython-input-1582754158.py:71: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['PatientID', 'pos'], as_index=False).apply(merge_rows).reset_index(drop=True)


In [16]:
# The v1 did not include the SeriesInstanceUID or the StudyDate
# We need to add back in manually - use combination of PatientID, SeriesNumber, and the SeriesDescription
# query for ones with missing SeriesInstanceUID

query = """
SELECT
  PatientID,
  StudyInstanceUID,
  StudyDate,
  SeriesNumber,
  SeriesInstanceUID,
  SeriesDescription,
  FrameOfReferenceUID
FROM
  `bigquery-public-data.idc_current.dicom_all`
WHERE
  Modality = 'MR' AND
  collection_id = 'prostatex'
ORDER BY
  PatientID
"""

df_selection = client_bq.query(query).to_dataframe()


In [17]:
df_prostatex = df.copy(deep=True)

In [18]:
# Now we take the SeriesInstanceUID field from the df_selection and insert into the df_prostatex

for index, row in df_prostatex.iterrows():
  # If SeriesInstanceUID is NaN, fill in
  DCMSerUID = row['SeriesInstanceUID']
  if pd.isna(DCMSerUID):
    # Get the corresponding PatientID (ProxID), SeriesNumber (DCMSerNum) and SeriesDescription (DCMSerDescr) to do the matching
    PatientID_tomatch = row['PatientID']
    SeriesNumber_tomatch = row['SeriesNumber']
    SeriesDescription_tomatch = row['SeriesDescription']
    # now find in the selection_df the corresponding SeriesInstanceUID
    # First we check the PatientID
    temp1_df = df_selection[df_selection['PatientID']==PatientID_tomatch]
    if (temp1_df.empty):
      print('PatientID: ' + str(PatientID_tomatch) + ' does not exist in IDC, skipping')
      continue
    else:
      temp2_df = temp1_df[temp1_df['SeriesNumber']==str(SeriesNumber_tomatch)]
      if (temp2_df.empty):
        print('PatientID: ' + str(PatientID_tomatch) + ' SeriesNumber: ' + str(SeriesNumber_tomatch) + ' do not exist in IDC, skipping')
        continue
      else:
        temp3_df = temp2_df[temp2_df['SeriesDescription']==str(SeriesDescription_tomatch)]
        if (temp3_df.empty):
          print('PatientID: ' + str(PatientID_tomatch) + ' SeriesNumber: ' + str(SeriesNumber_tomatch) + ' SeriesDescription' + str(SeriesDescription_tomatch) + ' do not exist in IDC, skipping')
        else:
          # get the SeriesInstanceUID
          SeriesInstanceUID = temp3_df['SeriesInstanceUID'].values[0]
          # assign value
          df_prostatex.loc[index, 'SeriesInstanceUID'] = SeriesInstanceUID


In [19]:
# Now we take the StudyDate field from the df_selection and insert into the df_prostatex
# Do the same with the StudyInstanceUID

for index, row in df_prostatex.iterrows():
  # Reassign all StudyDates!
  # Get the corresponding PatientID (ProxID), SeriesNumber (DCMSerNum) and SeriesDescription (DCMSerDescr) to do the matching
  PatientID_tomatch = row['PatientID']
  SeriesNumber_tomatch = row['SeriesNumber']
  SeriesDescription_tomatch = row['SeriesDescription']
  # now find in the selection_df the corresponding SeriesInstanceUID
  # First we check the PatientID
  temp1_df = df_selection[df_selection['PatientID']==PatientID_tomatch]
  if (temp1_df.empty):
    print('PatientID: ' + str(PatientID_tomatch) + ' does not exist in IDC, skipping')
    continue
  else:
    temp2_df = temp1_df[temp1_df['SeriesNumber']==str(SeriesNumber_tomatch)]
    if (temp2_df.empty):
      print('PatientID: ' + str(PatientID_tomatch) + ' SeriesNumber: ' + str(SeriesNumber_tomatch) + ' do not exist in IDC, skipping')
      continue
    else:
      temp3_df = temp2_df[temp2_df['SeriesDescription']==str(SeriesDescription_tomatch)]
      if (temp3_df.empty):
        print('PatientID: ' + str(PatientID_tomatch) + ' SeriesNumber: ' + str(SeriesNumber_tomatch) + ' SeriesDescription' + str(SeriesDescription_tomatch) + ' do not exist in IDC, skipping')
      else:
        # get the StudyDate
        StudyDate = temp3_df['StudyDate'].values[0]
        # assign value
        df_prostatex.loc[index, 'StudyDate'] = StudyDate
        # get the StudyInstanceUID
        StudyInstanceUID = temp3_df['StudyInstanceUID'].values[0]
        # assign the value
        df_prostatex.loc[index, 'StudyInstanceUID'] = StudyInstanceUID


In [20]:
# Remove rows with Nan in the SeriesInstanceUID

print('original number of rows in df_prostatex: ' + str(len(df_prostatex)))
df_prostatex = df_prostatex[df_prostatex['SeriesInstanceUID'].notna()]
print('new number of rows in df_prostatex: ' + str(len(df_prostatex)))

original number of rows in df_prostatex: 539
new number of rows in df_prostatex: 539


In [21]:
# Now add the FrameOfReferenceUID as a column

FrameOfReferenceUID_list = []
for n in range(0,len(df_prostatex)):
  SeriesInstanceUID = df_prostatex['SeriesInstanceUID'].values[n]
  FrameOfReferenceUID = df_selection[df_selection['SeriesInstanceUID']==SeriesInstanceUID]['FrameOfReferenceUID'].values[0]
  FrameOfReferenceUID_list.append(FrameOfReferenceUID)
df_prostatex['FrameOfReferenceUID'] = FrameOfReferenceUID_list

In [22]:
# Now we make the SeriesNumber and SeriesInstanceUID consistent

# Iterate through for each patient
patient_ids = sorted(list(set(df_prostatex['PatientID'].values)))
dcmsernum_list = []
dcmseruid_list = []
for n in range(0,len(patient_ids)):
  df_patient = df_prostatex[df_prostatex['PatientID']==patient_ids[n]]
  set_seriesnumber = list(df_patient['SeriesNumber'].values)[0]
  set_seriesuid = list(df_patient['SeriesInstanceUID'].values)[0]
  dcmsernum_list.append(set_seriesnumber)
  dcmseruid_list.append(set_seriesuid)

df_patient = pd.DataFrame()
df_patient['PatientID'] = patient_ids
df_patient['SeriesNumber'] = dcmsernum_list
df_patient['SeriesInstanceUID'] = dcmseruid_list

df_prostatex = df_prostatex[['PatientID',	'fid', 'pos', 'zone', 'clinsig', 'ggg', 'StudyDate', 'StudyInstanceUID', 'FrameOfReferenceUID']]
df_prostatex = pd.merge(left=df_prostatex, right=df_patient, how='inner', on=['PatientID'])


In [23]:
# Order the rows

df_prostatex = df_prostatex.sort_values(by=['PatientID', 'zone', 'fid'])

In [24]:
# Convert the clinsig to actual True/False, not strings
df_prostatex['clinsig'] = [item == "True" for item in df_prostatex['clinsig'].values]

# Convert Nan to None - at least for ggg
df_prostatex = df_prostatex.replace({np.nan: None})

# replace <NA> with None
df_prostatex = df_prostatex.replace({'<NA>': None})

# Convert ggg to integers, not strings
df_prostatex['ggg'] = [int(f) if type(f)==str else f for f in df_prostatex['ggg'].values]

df_prostatex = df_prostatex.replace({np.nan: None})

In [25]:
df_prostatex.head(10)

,PatientID,fid,pos,zone,clinsig,ggg,StudyDate,StudyInstanceUID,FrameOfReferenceUID,SeriesNumber,SeriesInstanceUID
0,ProstateX-0000,1,25.7457 31.8707 -38.511,PZ,True,3.0,2011-07-07,1.3.6.1.4.1.14519.5.2.1.7311.5101.158323547117...,1.3.6.1.4.1.14519.5.2.1.7311.5101.167989189699...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.160028252338...
1,ProstateX-0001,1,-40.5367071921656 29.320722668457 -16.70766907...,AS,False,1.0,2011-07-08,1.3.6.1.4.1.14519.5.2.1.7311.5101.204786391199...,1.3.6.1.4.1.14519.5.2.1.7311.5101.410957498702...,10,1.3.6.1.4.1.14519.5.2.1.7311.5101.296510477295...
3,ProstateX-0002,1,-27.0102 41.5467 -26.0469,PZ,True,2.0,2011-07-15,1.3.6.1.4.1.14519.5.2.1.7311.5101.170110802438...,1.3.6.1.4.1.14519.5.2.1.7311.5101.216292633397...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.479812804428...
2,ProstateX-0002,2,-2.058 38.6752 -34.6104,PZ,False,1.0,2011-07-15,1.3.6.1.4.1.14519.5.2.1.7311.5101.170110802438...,1.3.6.1.4.1.14519.5.2.1.7311.5101.216292633397...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.479812804428...
5,ProstateX-0003,1,22.1495 31.2717 -2.45933,TZ,False,None,2011-10-17,1.3.6.1.4.1.14519.5.2.1.7311.5101.879702148497...,1.3.6.1.4.1.14519.5.2.1.7311.5101.210593189161...,3,1.3.6.1.4.1.14519.5.2.1.7311.5101.320508777856...
4,ProstateX-0003,2,-21.2871 19.3995 19.7429,TZ,False,None,2011-10-17,1.3.6.1.4.1.14519.5.2.1.7311.5101.879702148497...,1.3.6.1.4.1.14519.5.2.1.7311.5101.210593189161...,3,1.3.6.1.4.1.14519.5.2.1.7311.5101.320508777856...
6,ProstateX-0004,1,-7.69665 3.64226 23.1659,AS,False,1.0,2011-10-18,1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612...,1.3.6.1.4.1.14519.5.2.1.7311.5101.283465068568...,5,1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270...
7,ProstateX-0005,0,-14.5174331665039 49.4428329467773 20.78152465...,PZ,True,2.0,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...
9,ProstateX-0005,1,-38.6276 42.2781 21.4084,PZ,True,3.0,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...
8,ProstateX-0005,1,-22.0892639160156 25.4668045043945 22.87915420...,TZ,False,None,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...


In [26]:
# Now we do some sanity checking
# We should only have as many series as patients

num_rows = len(df_prostatex)
print('num_rows: ' + str(num_rows))

# This is the number of folders we should have in the bucket!
num_patients = len(list(set(df_prostatex['PatientID'].values)))
print('num_patients: ' + str(num_patients))

# This is the number of SRs that should be created!
num_series = len(list(set(df_prostatex['SeriesInstanceUID'].values)))
print("num_series: " + str(num_series))

# This is the number of SRs that should be created!
num_frame = len(list(set(df_prostatex['FrameOfReferenceUID'].values)))
print("num_frame: " + str(num_frame))

num_rows: 539
num_patients: 345
num_series: 345
num_frame: 345


In [27]:
# Sanity checking continued
# Check which patients are missing

patient_ids = sorted(list(set(df_prostatex['PatientID'].values)))
query = """
SELECT
  DISTINCT(PatientID)
FROM
  index
WHERE
  Modality = 'MR' AND
  collection_id = 'prostatex'
ORDER BY
  PatientID
"""
patient_df = client.sql_query(query)
patient_ids_total = sorted(list(patient_df['PatientID'].values))
patient_ids_missing = list(set(patient_ids_total) - set(patient_ids))
print('patient_ids_missing: ' + str(patient_ids_missing))

# ProstateX-0325 is missing any clincal metadata in the tables. This is correct.

patient_ids_missing: ['ProstateX-0325']


In [35]:
# Now we need to reassign the fid for each Study
# Sometimes they overlap

df_prostatex['TrackingIdentifier'] = df_prostatex.groupby('StudyInstanceUID').cumcount()+1
df_prostatex.head(10)

,PatientID,fid,pos,zone,clinsig,ggg,StudyDate,StudyInstanceUID,FrameOfReferenceUID,SeriesNumber,SeriesInstanceUID,TrackingIdentifier
0,ProstateX-0000,1,25.7457 31.8707 -38.511,PZ,True,3.0,2011-07-07,1.3.6.1.4.1.14519.5.2.1.7311.5101.158323547117...,1.3.6.1.4.1.14519.5.2.1.7311.5101.167989189699...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.160028252338...,1
1,ProstateX-0001,1,-40.5367071921656 29.320722668457 -16.70766907...,AS,False,1.0,2011-07-08,1.3.6.1.4.1.14519.5.2.1.7311.5101.204786391199...,1.3.6.1.4.1.14519.5.2.1.7311.5101.410957498702...,10,1.3.6.1.4.1.14519.5.2.1.7311.5101.296510477295...,1
3,ProstateX-0002,1,-27.0102 41.5467 -26.0469,PZ,True,2.0,2011-07-15,1.3.6.1.4.1.14519.5.2.1.7311.5101.170110802438...,1.3.6.1.4.1.14519.5.2.1.7311.5101.216292633397...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.479812804428...,1
2,ProstateX-0002,2,-2.058 38.6752 -34.6104,PZ,False,1.0,2011-07-15,1.3.6.1.4.1.14519.5.2.1.7311.5101.170110802438...,1.3.6.1.4.1.14519.5.2.1.7311.5101.216292633397...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.479812804428...,2
5,ProstateX-0003,1,22.1495 31.2717 -2.45933,TZ,False,None,2011-10-17,1.3.6.1.4.1.14519.5.2.1.7311.5101.879702148497...,1.3.6.1.4.1.14519.5.2.1.7311.5101.210593189161...,3,1.3.6.1.4.1.14519.5.2.1.7311.5101.320508777856...,1
4,ProstateX-0003,2,-21.2871 19.3995 19.7429,TZ,False,None,2011-10-17,1.3.6.1.4.1.14519.5.2.1.7311.5101.879702148497...,1.3.6.1.4.1.14519.5.2.1.7311.5101.210593189161...,3,1.3.6.1.4.1.14519.5.2.1.7311.5101.320508777856...,2
6,ProstateX-0004,1,-7.69665 3.64226 23.1659,AS,False,1.0,2011-10-18,1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612...,1.3.6.1.4.1.14519.5.2.1.7311.5101.283465068568...,5,1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270...,1
7,ProstateX-0005,0,-14.5174331665039 49.4428329467773 20.78152465...,PZ,True,2.0,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...,1
9,ProstateX-0005,1,-38.6276 42.2781 21.4084,PZ,True,3.0,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...,2
8,ProstateX-0005,1,-22.0892639160156 25.4668045043945 22.87915420...,TZ,False,None,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...,3


In [28]:
# Save out the csv for easy checking later

df_prostatex.to_csv("/content/SR_ProstateX.csv")

# Create SRs

In [36]:
def save_SR_for_case(findings_df, reference_dcm_file, output_filename):
  """
  findings_df        - a dataframe containing the findings of a single series
  reference_dcm_file - path to a single dcm file to be used as a reference
  output_filename    - output filename for the SR
  """

  # Read dcm file
  image_dataset = dcmread(reference_dcm_file)

  # Here we delete the PertinentOtherEvidenceSequence
  # So that it is not referenced in the evidence when we create the Comprehensive3DSR
  # We do not need it since we are using SCOORD3D points, and should only use the FrameOfReferenceUID
  if "PertinentOtherEvidenceSequence" in image_dataset:
    try:
      del image_dataset.PertinentOtherEvidenceSequence
    except:
      print("Cannot delete the PertinentOtherEvidenceSequence from the pydicom dataset")

  # Get the FrameOfReferenceUID
  FrameOfReferenceUID = pydicom.dcmread(reference_dcm_file)['FrameOfReferenceUID'].value
  print('FrameOfReferenceUID: ' + str(FrameOfReferenceUID))

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  observer_person_context = ObserverContext(
      observer_type=codes.DCM.Person,
      observer_identifying_attributes=PersonObserverIdentifyingAttributes(
          name='Anonymous^Reader'
      )
  )
  observer_device_context = ObserverContext(
      observer_type=codes.DCM.Device,
      observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
          uid=generate_uid()
      )
  )
  observation_context = ObservationContext(
      observer_person_context=observer_person_context,
      #observer_device_context=observer_device_context,
  )

  # Add in the clinical information
  ClinicalTrialProtocolID = "doi:10.5281/zenodo.15643312"
  IssuerOfClinicalTrialProtocolID = "DOI"
  ClinicalTrialSponsorName = "National Cancer Institute"
  # added
  ClinicalTrialProtocolName = ""
  ClinicalTrialSiteID = ""
  ClinicalTrialSiteName = ""
  PatientID = findings_df['PatientID'].values[0]
  ClinicalTrialSubjectID = PatientID
  ClinicalTrialSubjectReadingID = PatientID

  imaging_measurements = []

  for index,one_finding in findings_df.iterrows():

    print("Parsing "+one_finding['pos'])

    # finding_id = one_finding['fid']
    one_tracking_identifier = one_finding['TrackingIdentifier']

    pos_items = re.search(r"\ ?(-?[0-9]*\.[0-9]*)\ +(-?[0-9]*\.[0-9]*)\ +(-?[0-9]*\.[0-9]*)", one_finding['pos'])
    if not pos_items:
      print("Failed to parse "+one_finding['pos'])

    referenced_region_3d = ImageRegion3D(
        graphic_type=GraphicTypeValues3D.POINT,
        graphic_data=np.array([[float(pos_items.group(1)), float(pos_items.group(2)), float(pos_items.group(3))]]),
        #source_image=source_image_ref
        frame_of_reference_uid=str(FrameOfReferenceUID)
    )

    # Describe the anatomic site at which observations were made
    if one_finding['zone'] == 'TZ':
      finding_location = CodedConcept(
                    value="399384005",
                    meaning="Transition zone of the prostate",
                    scheme_designator="SCT"
                )
    elif one_finding['zone'] == 'PZ':
      finding_location = CodedConcept(
                    value="279706003",
                    meaning="Peripheral zone of the prostate",
                    scheme_designator="SCT"
                )
    elif one_finding['zone'] == 'AS':
      finding_location = CodedConcept(
                    value="717025007",
                    meaning="Anterior fibromuscular stroma of prostate",
                    scheme_designator="SCT"
                )
    elif one_finding['zone'] == 'SV':
      finding_location = CodedConcept(
                    value="64739004",
                    meaning="Seminal vesicle",
                    scheme_designator="SCT"
                )

    finding_sites = [
        FindingSite(anatomic_location=finding_location)
    ]

    evaluation = []
    ggg_code = None

    if one_finding['clinsig'] == True:
        evaluation=[QualitativeEvaluation(
                name=CodedConcept(
                    value="RID49502",
                    meaning="clinically significant prostate cancer",
                    scheme_designator="RADLEX"
                ),
                value=CodedConcept(value='373066001', scheme_designator='SCT', meaning='Yes', scheme_version=None)
            )]

    elif one_finding['clinsig'] == False:
        evaluation=[QualitativeEvaluation(
                name=CodedConcept(
                    value="RID49502",
                    meaning="clinically significant prostate cancer",
                    scheme_designator="RADLEX"
                ),
                value=CodedConcept(value='373067005', scheme_designator='SCT', meaning='No', scheme_version=None)
            )]

    if one_finding['ggg'] is not None:
          if one_finding['ggg'] == 1:
              evaluation.append(QualitativeEvaluation(
                  name=CodedConcept(
                      value="385377005",
                      meaning="Gleason grade finding for prostatic cancer", # taken from SNOMED browser.
                      scheme_designator="SCT"
                  ),
                  value=CodedConcept(value='860742008', scheme_designator='SCT', meaning='Gleason grade group 1', scheme_version=None)
              )
              )

          elif one_finding['ggg'] == 2:
              evaluation.append(QualitativeEvaluation(
                  name=CodedConcept(
                      value="385377005",
                      meaning="Gleason grade finding for prostatic cancer",
                      scheme_designator="SCT"
                  ),
                  value=CodedConcept(value='860743003', scheme_designator='SCT', meaning='Gleason grade group 2', scheme_version=None)
              )
              )

          elif one_finding['ggg'] == 3:
              evaluation.append(QualitativeEvaluation(
                  name=CodedConcept(
                      value="385377005",
                      meaning="Gleason grade finding for prostatic cancer",
                      scheme_designator="SCT"
                  ),
                  value=CodedConcept(value='860744009', scheme_designator='SCT', meaning='Gleason grade group 3', scheme_version=None)
              )
              )

          elif one_finding['ggg'] == 4:
              evaluation.append(QualitativeEvaluation(
                  name=CodedConcept(
                      value="385377005",
                      meaning="Gleason grade finding for prostatic cancer",
                      scheme_designator="SCT"
                  ),
                  value=CodedConcept(value='860745005', scheme_designator='SCT', meaning='Gleason grade group 4', scheme_version=None)
              )
              )

          elif one_finding['ggg'] == 5:
              evaluation.append(QualitativeEvaluation(
                  name=CodedConcept(
                      value="385377005",
                      meaning="Gleason grade finding for prostatic cancer",
                      scheme_designator="SCT"
                  ),
                  value=CodedConcept(value='860746006', scheme_designator='SCT', meaning='Gleason grade group 5', scheme_version=None)
              )
              )

    imaging_measurements.append(
        PlanarROIMeasurementsAndQualitativeEvaluations(
            tracking_identifier=TrackingIdentifier(
                uid=generate_uid(),
                identifier=str(one_tracking_identifier)
            ),
            referenced_region=referenced_region_3d,
            finding_type=codes.SCT.Lesion,
            #measurements=measurements,
            qualitative_evaluations=evaluation,
            finding_sites=finding_sites
        )
    )

  print(len(imaging_measurements))

  # Create the report content
  procedure_code = CodedConcept(value="719178004", scheme_designator="SCT",
                                # meaning="Multiparametric magnetic resonance imaging of prostate")
                                meaning="Multiparametric MRI of prostate")
  measurement_report = MeasurementReport(
      observation_context=observation_context,
      procedure_reported=procedure_code,
      imaging_measurements=imaging_measurements
  )

  # Create the Structured Report instance
  sr_dataset = Comprehensive3DSR(
      evidence=[image_dataset],
      content=measurement_report[0],
      series_number=findings_df['SeriesNumber'].values[0].astype(np.int32) + 100,
      series_instance_uid=generate_uid(),
      series_description="Prostate lesion target annotations",
      sop_instance_uid=generate_uid(),
      instance_number=1,
      manufacturer='IDC',
      is_complete = True,
      is_final=True
  )

  # Add clinical dataset information
  sr_dataset.ClinicalTrialProtocolID=ClinicalTrialProtocolID,
  sr_dataset.IssuerOfClinicalTrialProtocolID=IssuerOfClinicalTrialProtocolID,
  sr_dataset.ClinicalTrialSponsorName=ClinicalTrialSponsorName
  # added
  sr_dataset.ClinicalTrialProtocolName = ClinicalTrialProtocolName
  sr_dataset.ClinicalTrialSiteID = ClinicalTrialSiteID
  sr_dataset.ClinicalTrialSiteName = ClinicalTrialSiteName
  sr_dataset.ClinicalTrialSubjectID = ClinicalTrialSubjectID
  sr_dataset.ClinicalTrialSubjectReadingID = ClinicalTrialSubjectReadingID

  # Save out the SR
  sr_dataset.save_as(output_filename, enforce_file_format=True)

  return


In [37]:
sr_directory = "/content/sr"
if not os.path.isdir(sr_directory):
  os.mkdir(sr_directory)

PatientID_list = sorted(list(set(df_prostatex['PatientID'].values)))
num_patients = len(PatientID_list)
print('num_patients: ' + str(num_patients))

num_patients: 345


In [38]:
# check that ProstateX-0000 has clinsig and ggg - 1 point
df_temp = df_prostatex[df_prostatex['PatientID']=="ProstateX-0005"]
df_temp

,PatientID,fid,pos,zone,clinsig,ggg,StudyDate,StudyInstanceUID,FrameOfReferenceUID,SeriesNumber,SeriesInstanceUID,TrackingIdentifier
7,ProstateX-0005,0,-14.5174331665039 49.4428329467773 20.78152465...,PZ,True,2.0,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...,1
9,ProstateX-0005,1,-38.6276 42.2781 21.4084,PZ,True,3.0,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...,2
8,ProstateX-0005,1,-22.0892639160156 25.4668045043945 22.87915420...,TZ,False,None,2011-10-20,1.3.6.1.4.1.14519.5.2.1.7311.5101.247316591887...,1.3.6.1.4.1.14519.5.2.1.7311.5101.253068539461...,4,1.3.6.1.4.1.14519.5.2.1.7311.5101.119114186762...,3


In [40]:
################################################################################
################################################################################

sr_directory = "/content/sr"
if not os.path.isdir(sr_directory):
  os.mkdir(sr_directory)

PatientID_list = sorted(list(set(df_prostatex['PatientID'].values)))
num_patients = len(PatientID_list)
print('num_patients: ' + str(num_patients))

########################
### For each patient ###
########################

checkpoints = {int(len(PatientID_list) * i / 10) for i in range(1, 11)}

for index,PatientID in enumerate(PatientID_list,1):

  print('********** On patient: ' + str(PatientID) + ' **********')

  df_prostatex_patient = df_prostatex[df_prostatex['PatientID']==PatientID]
  SeriesInstanceUID_list = sorted(list(set(df_prostatex_patient['SeriesInstanceUID'].values)))
  num_series = len(SeriesInstanceUID_list)
  print('num_series: ' + str(num_series))

  #######################
  ### For each series ###
  #######################

  for SeriesInstanceUID in SeriesInstanceUID_list:

    print('***** On series: ' + str(SeriesInstanceUID) + ' *****')
    df_prostatex_series = df_prostatex_patient[df_prostatex_patient['SeriesInstanceUID']==SeriesInstanceUID]

    # set output filename for SR
    output_filename = os.path.join(sr_directory, PatientID + '.dcm')
    # set output filename for SR in bucket
    output_filename_bucket = os.path.join(bucket_sr_directory, PatientID + '.dcm')

    ### get the df ###

    # # Get the specific row of the df
    # findings_df = df_prostatex_series.iloc[n]
    # # Convert the Series to a DataFrame
    # findings_df = findings_df.to_frame().T
    # # Reset index of the new DataFrame
    # findings_df = findings_df.reset_index(drop=True)
    findings_df = df_prostatex_series.copy(deep=True)

    ### get the reference dcm ###

    # get the SeriesInstanceUID
    SeriesInstanceUID = findings_df['SeriesInstanceUID'].values[0]
    print('SeriesInstanceUID: ' + str(SeriesInstanceUID))

    # create a temporary dicom_directory
    dicom_directory = "/content/data"
    if not os.path.isdir(dicom_directory):
      os.mkdir(dicom_directory)

    # download the series
    client.download_from_selection(
      seriesInstanceUID=SeriesInstanceUID,
      downloadDir=dicom_directory,
      dirTemplate=''
    )

    # get one file
    reference_dcm_file = os.path.join(dicom_directory, os.listdir(dicom_directory)[0])

    # create and save the SR
    save_SR_for_case(findings_df, reference_dcm_file, output_filename)

    # copy the SR to the bucket
    if os.path.exists(output_filename):
      try:
        !gsutil cp $output_filename $output_filename_bucket
      except:
        print('ERROR: cannot copy from ' + str(output_filename) + ' to: ' + str(output_filename_bucket))

    # delete files from dicom_directory
    !rm -r $dicom_directory

    if index in checkpoints:
      print(f"{(index / len(PatientID_list)) * 100:.0f}% of patients processed.")

num_patients: 345
********** On patient: ProstateX-0000 **********
num_series: 1
***** On series: 1.3.6.1.4.1.14519.5.2.1.7311.5101.160028252338004527274326500702 *****
SeriesInstanceUID: 1.3.6.1.4.1.14519.5.2.1.7311.5101.160028252338004527274326500702


/usr/local/lib/python3.12/dist-packages/highdicom/base.py:181: UserWarning: The string "ProstateX-0000" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  check_person_name(patient_name)
/usr/local/lib/python3.12/dist-packages/pydicom/valuerep.py:440: UserWarning: A value of type 'tuple' cannot be assigned to a tag with VR LO.
  warn_and_log(msg)


FrameOfReferenceUID: 1.3.6.1.4.1.14519.5.2.1.7311.5101.167989189699391273938467151375
Parsing 25.7457 31.8707 -38.511
1
Copying file:///content/sr/ProstateX-0000.dcm [Content-Type=application/dicom]...
/ [1 files][  5.3 KiB/  5.3 KiB]                                                
Operation completed over 1 objects/5.3 KiB.                                      
********** On patient: ProstateX-0001 **********
num_series: 1
***** On series: 1.3.6.1.4.1.14519.5.2.1.7311.5101.296510477295449267377313817541 *****
SeriesInstanceUID: 1.3.6.1.4.1.14519.5.2.1.7311.5101.296510477295449267377313817541


/usr/local/lib/python3.12/dist-packages/highdicom/base.py:181: UserWarning: The string "ProstateX-0001" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  check_person_name(patient_name)
/usr/local/lib/python3.12/dist-packages/pydicom/valuerep.py:440: UserWarning: A value of type 'tuple' cannot be assigned to a tag with VR LO.
  warn_and_log(msg)


FrameOfReferenceUID: 1.3.6.1.4.1.14519.5.2.1.7311.5101.410957498702420257441538976223
Parsing -40.5367071921656 29.320722668457 -16.7076690704346
1
Copying file:///content/sr/ProstateX-0001.dcm [Content-Type=application/dicom]...
/ [1 files][  5.3 KiB/  5.3 KiB]                                                
Operation completed over 1 objects/5.3 KiB.                                      
********** On patient: ProstateX-0002 **********
num_series: 1
***** On series: 1.3.6.1.4.1.14519.5.2.1.7311.5101.479812804428819225709948044920 *****
SeriesInstanceUID: 1.3.6.1.4.1.14519.5.2.1.7311.5101.479812804428819225709948044920


...

FrameOfReferenceUID: 1.3.6.1.4.1.14519.5.2.1.7310.5101.196819474303715527407522352952
Parsing 0.553015 18.7555 18.5042
1
Copying file:///content/sr/ProstateX-0343.dcm [Content-Type=application/dicom]...
/ [1 files][  5.8 KiB/  5.8 KiB]                                                
Operation completed over 1 objects/5.8 KiB.                                      
********** On patient: ProstateX-0344 **********
num_series: 1
***** On series: 1.3.6.1.4.1.14519.5.2.1.7310.5101.284671843735327551266044347858 *****
SeriesInstanceUID: 1.3.6.1.4.1.14519.5.2.1.7310.5101.284671843735327551266044347858


/usr/local/lib/python3.12/dist-packages/highdicom/base.py:181: UserWarning: The string "ProstateX-0344" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  check_person_name(patient_name)
/usr/local/lib/python3.12/dist-packages/pydicom/valuerep.py:440: UserWarning: A value of type 'tuple' cannot be assigned to a tag with VR LO.
  warn_and_log(msg)


FrameOfReferenceUID: 1.3.6.1.4.1.14519.5.2.1.7310.5101.139348806503640763364411511158
Parsing 19.3829 55.2417 3.26941
Parsing 8.86952 34.8962 -3.25847
2
Copying file:///content/sr/ProstateX-0344.dcm [Content-Type=application/dicom]...
/ [1 files][  7.1 KiB/  7.1 KiB]                                                
Operation completed over 1 objects/7.1 KiB.                                      
********** On patient: ProstateX-0345 **********
num_series: 1
***** On series: 1.3.6.1.4.1.14519.5.2.1.7310.5101.234925765587533703261014383053 *****
SeriesInstanceUID: 1.3.6.1.4.1.14519.5.2.1.7310.5101.234925765587533703261014383053


/usr/local/lib/python3.12/dist-packages/highdicom/base.py:181: UserWarning: The string "ProstateX-0345" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  check_person_name(patient_name)
/usr/local/lib/python3.12/dist-packages/pydicom/valuerep.py:440: UserWarning: A value of type 'tuple' cannot be assigned to a tag with VR LO.
  warn_and_log(msg)


FrameOfReferenceUID: 1.3.6.1.4.1.14519.5.2.1.7310.5101.220673677138630730663741100306
Parsing -16.2035 35.3915 5.65363
Parsing 8.59292 15.8902 10.8248
2
Copying file:///content/sr/ProstateX-0345.dcm [Content-Type=application/dicom]...
/ [1 files][  7.1 KiB/  7.1 KiB]                                                
Operation completed over 1 objects/7.1 KiB.                                      
100% of patients processed.


In [34]:
findings_df['StudyInstanceUID'].values[0]

'1.3.6.1.4.1.14519.5.2.1.7311.5101.158323547117540061132729905711'